<a href="https://colab.research.google.com/github/epadam/Machine-Learning-Tutorial-Demo-Resources/blob/master/notebooks/nlp/grammer%20classification/Cola_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [4]:
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments, pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn import model_selection
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from datasets import load_dataset
data = load_dataset('glue', 'cola')

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_data = data['train']
validation_data = data['validation']
test_data = data['test']

In [6]:
train_data[0]

{'idx': 0,
 'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
train_data = train_data.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512, padding='max_length'), batched=True)
validation_data = validation_data.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512, padding='max_length'), batched=True)
test_data = test_data.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512, padding='max_length'), batched=True)


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
train_data.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'idx': Value(dtype='int32', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': ClassLabel(num_classes=2, names=['unacceptable', 'acceptable'], names_file=None, id=None),
 'sentence': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [17]:
def metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return{
    'accuracy': acc,
    'f1': f1,
    'precision': precision,
    'recall': recall
  }

In [18]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate =3e-4,
    num_train_epochs =2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir = './log',
    logging_steps = 100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    evaluation_strategy='steps'
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [19]:
trainer =Trainer(
    model=model,
    args=training_args,
    train_dataset= train_data,
    eval_dataset = validation_data,
    compute_metrics=metrics
)

In [20]:
train_out = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 8551
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2138


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.650800,0.640031,0.691275,0.408730,0.345638,0.500000
200,0.646800,0.658514,0.691275,0.408730,0.345638,0.500000
300,0.635400,0.639246,0.691275,0.408730,0.345638,0.500000
400,0.621900,0.618540,0.691275,0.408730,0.345638,0.500000
500,0.611600,0.620171,0.691275,0.408730,0.345638,0.500000
600,0.633400,0.618599,0.691275,0.408730,0.345638,0.500000
700,0.640100,0.618320,0.691275,0.408730,0.345638,0.500000
800,0.651100,0.621301,0.691275,0.408730,0.345638,0.500000
900,0.621000,0.636229,0.691275,0.408730,0.345638,0.500000
1000,0.611900,0.618185,0.691275,0.408730,0.345638,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results/checkpoint-100
Configuration saved in ./results/checkpoint-100/config.json
Model weights saved in ./results/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/

In [21]:
torch.save(model, './model.pt')

In [22]:
model = model.cpu()

In [24]:
classifier = pipeline( 'text-classification', model=model, tokenizer=tokenizer)

In [35]:
result = classifier(test_data[0]['sentence'])

test_data[36]['sentence'], result

('Spray the paint onto the whole wall completely.',
 [{'label': 'LABEL_1', 'score': 0.6416943073272705}])